# What we'll want to do
- Download the stock price history, calculate percentage of growth for each day
- Fetch all the data
    - Download from Twitter, Reddit, Reuters and Yahoo Finance
    - Perform sentiment analysis on the single 


## Possible features
- Qty of positive negative neutral tweets/reddit comments/articles
- Starting price
- Last day percentage


TODO
- Adapt the text RNN to these texts
- y will be the percentage of growth
- repeat this for a set of firms
- get the highest percentages of growth

To construct the dataset:
- Define a function that gets the percentage (starting price-closing price / starting) given a stock symbol and a day
- Define a function that calls the latter and iterates over N days
- Define 3 functions that get tweets/articles/reddit comments given a stock symbol and a day (this will use a JSON containing the search terms)
- Define a function that calls the previous ones, creates a dataset of texts->stock changing

## API Key for AlphaVantage
`XWJ7JPW4QLVIHMTU`


In [1]:
import pandas as pd

In [2]:
API_KEY = "XWJ7JPW4QLVIHMTU"
API_FUNCTION = "TIME_SERIES_DAILY_ADJUSTED"
OUTPUT_SIZE = "full"
N_DAYS = 1000

In [3]:
def get_stock_dataframe(symbol):
    endpoint = "https://www.alphavantage.co/query?function={}&symbol={}&outputsize={}&apikey={}&datatype=csv".format(API_FUNCTION,symbol, OUTPUT_SIZE, API_KEY)
    dataframe = pd.read_csv(endpoint)
    dataframe["change"] = (dataframe['close']-dataframe['open'])/dataframe['open']
    dataframe = dataframe.head(3000) # No ancient shit please
    return dataframe

In [4]:
!pip3 install --force-reinstall git+https://github.com/twintproject/twint.git@origin/master#egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /private/var/folders/yl/3bwgxtvd3d55k4h9wn4c996w0000gn/T/pip-install-qhzegbzj/twint
  Using cached aiohttp-3.7.2-cp36-cp36m-macosx_10_14_x86_64.whl (649 kB)
  Using cached aiodns-2.0.0-py2.py3-none-any.whl (4.8 kB)
  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached cchardet-2.1.7-cp36-cp36m-macosx_10_9_x86_64.whl (124 kB)
  Using cached dataclasses-0.7-py3-none-any.whl (18 kB)
  Using cached elasticsearch-7.9.1-py2.py3-none-any.whl (219 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached pandas-1.1.4-cp36-cp36m-macosx_10_9_x86_64.whl (10.2 MB)
  Using cached aiohttp_socks-0.5.5-py3-none-any.whl (9.1 kB)
  Using cached schedule-0.6.0-py2.py3-none-any.whl (8.7 kB)
  Using cached geopy-2.0.0-py3-none-any.whl (111 kB)
Processing /Users/simone/Library/Caches/pip/wheels/12/8a/a1/0ab79c1c2c58aa7647d0dea17f2af2358b65e1eecac8094b5e/fake_useragent-0.1.11-py3-none-any.

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4
  Attempting uninstall: pytz
    Found existing installation: pytz 2020.4
    Uninstalling pytz-2020.4:
      Successfully uninstalled pytz-2020.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.4
    Uninstalling pandas-1.1.4:
      Successfully uninstalled pandas-1.1.4
  Attempting uninstall: python-socks
    Found existing installation: python-socks 1.1.0
    Uninstalling python-socks-1.1.0:
      Successfully uninstalled python-socks-1.1.0
  Attempting uninstall: aiohttp-socks
    Found existing installation: aiohttp-socks 0.5.5
    Uninstalling aiohttp-socks-0.5.5:
      Successfully uninstalled aiohttp-socks-0.5.5
  Attempting uninstall: schedule
    Found existing installation: schedule 0.6.0
    Uninstalling schedule-0.6.0:
      Successfully uninstalled schedule-0.6.0
  Attempting uninstall: geographic

In [5]:
import emoji

def clean_text_from_emoji(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])

    return clean_text

In [6]:
import re

def clean_text(text):
    text = re.sub(r'https?:\/\/\S*[\s]', '', text, flags=re.MULTILINE)
    text = clean_text_from_emoji(text)
    return text

In [30]:
import os
import re
import subprocess

def get_tweets(keywords, date):
    # Getting the day for the Since parameter in tweets download
    date_split = date.split('-')
    date_before = int(date_split[2])-1
    date_before_string = date_split[0]+'-'+date_split[1]+'-'+str(date_before)
    result = []
    for keyword in keywords:
        print("Fetching keyword {} for day {}".format(keyword, date))
        filename = "{}-{}.csv".format(keyword, date)
        if not os.path.exists(filename):
            command = "twint -s '{}' -o {}-{}.csv --csv --since '{}' --until '{}' --limit 1 --lang en".format(keyword, keyword, date, date_before_string, date)
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
            process.wait()
        if os.path.exists(filename):
            tweet_data = pd.read_csv(filename, delimiter="\t")
            result.append(tweet_data["tweet"])
    if len(result)>0:
        full_df = pd.concat(result, ignore_index=True)
        full_df.drop_duplicates()
        out = ' '.join(full_df)
        out = clean_text(out)
    else:
        out = ""
    return out


In [28]:
import json
with open("firm_data.json") as json_file: 
    firm_data = json.loads(json_file.read())

In [10]:
firm_data[0]

{'firm': 'Apple',
 'related_terms': ['iPhone', 'iPad', 'Macbook', 'iOS', 'macOS'],
 'stock_code': 'AAPL',
 'ordered': 0}

In [ ]:
data = {}
for firm in firm_data[:3]:
    money = get_stock_dataframe(firm["stock_code"])
    money = money.head(N_DAYS)
    money['tweets']=money.apply(lambda row: get_tweets(firm_data[0]["related_terms"], row["timestamp"]), axis=1)
    data[firm["firm"]]=money[["tweets", "change"]]

Fetching keyword iPhone for day 2020-11-09
Fetching keyword iPad for day 2020-11-09
Fetching keyword Macbook for day 2020-11-09
Fetching keyword iOS for day 2020-11-09
Fetching keyword macOS for day 2020-11-09
Fetching keyword iPhone for day 2020-11-06
Fetching keyword iPad for day 2020-11-06
Fetching keyword Macbook for day 2020-11-06
Fetching keyword iOS for day 2020-11-06
Fetching keyword macOS for day 2020-11-06
Fetching keyword iPhone for day 2020-11-05
Fetching keyword iPad for day 2020-11-05
Fetching keyword Macbook for day 2020-11-05
Fetching keyword iOS for day 2020-11-05
Fetching keyword macOS for day 2020-11-05
Fetching keyword iPhone for day 2020-11-04
Fetching keyword iPad for day 2020-11-04
Fetching keyword Macbook for day 2020-11-04
Fetching keyword iOS for day 2020-11-04
Fetching keyword macOS for day 2020-11-04
Fetching keyword iPhone for day 2020-11-03
Fetching keyword iPad for day 2020-11-03
Fetching keyword Macbook for day 2020-11-03
Fetching keyword iOS for day 202

Fetching keyword iPad for day 2020-09-15
Fetching keyword Macbook for day 2020-09-15
Fetching keyword iOS for day 2020-09-15
Fetching keyword macOS for day 2020-09-15
Fetching keyword iPhone for day 2020-09-14
Fetching keyword iPad for day 2020-09-14
Fetching keyword Macbook for day 2020-09-14
Fetching keyword iOS for day 2020-09-14
Fetching keyword macOS for day 2020-09-14
Fetching keyword iPhone for day 2020-09-11
Fetching keyword iPad for day 2020-09-11
Fetching keyword Macbook for day 2020-09-11
Fetching keyword iOS for day 2020-09-11
Fetching keyword macOS for day 2020-09-11
Fetching keyword iPhone for day 2020-09-10
Fetching keyword iPad for day 2020-09-10
Fetching keyword Macbook for day 2020-09-10
Fetching keyword iOS for day 2020-09-10
Fetching keyword macOS for day 2020-09-10
Fetching keyword iPhone for day 2020-09-09
Fetching keyword iPad for day 2020-09-09
Fetching keyword Macbook for day 2020-09-09
Fetching keyword iOS for day 2020-09-09
Fetching keyword macOS for day 2020

Fetching keyword Macbook for day 2020-07-21
Fetching keyword iOS for day 2020-07-21
Fetching keyword macOS for day 2020-07-21
Fetching keyword iPhone for day 2020-07-20
Fetching keyword iPad for day 2020-07-20
Fetching keyword Macbook for day 2020-07-20
Fetching keyword iOS for day 2020-07-20
Fetching keyword macOS for day 2020-07-20
Fetching keyword iPhone for day 2020-07-17
Fetching keyword iPad for day 2020-07-17
Fetching keyword Macbook for day 2020-07-17
Fetching keyword iOS for day 2020-07-17
Fetching keyword macOS for day 2020-07-17
Fetching keyword iPhone for day 2020-07-16
Fetching keyword iPad for day 2020-07-16
Fetching keyword Macbook for day 2020-07-16
Fetching keyword iOS for day 2020-07-16
Fetching keyword macOS for day 2020-07-16
Fetching keyword iPhone for day 2020-07-15
Fetching keyword iPad for day 2020-07-15
Fetching keyword Macbook for day 2020-07-15
Fetching keyword iOS for day 2020-07-15
Fetching keyword macOS for day 2020-07-15
Fetching keyword iPhone for day 20

Fetching keyword iOS for day 2020-05-26
Fetching keyword macOS for day 2020-05-26
Fetching keyword iPhone for day 2020-05-22
Fetching keyword iPad for day 2020-05-22
Fetching keyword Macbook for day 2020-05-22
Fetching keyword iOS for day 2020-05-22
Fetching keyword macOS for day 2020-05-22
Fetching keyword iPhone for day 2020-05-21
Fetching keyword iPad for day 2020-05-21
Fetching keyword Macbook for day 2020-05-21
Fetching keyword iOS for day 2020-05-21
Fetching keyword macOS for day 2020-05-21
Fetching keyword iPhone for day 2020-05-20
Fetching keyword iPad for day 2020-05-20
Fetching keyword Macbook for day 2020-05-20
Fetching keyword iOS for day 2020-05-20
Fetching keyword macOS for day 2020-05-20
Fetching keyword iPhone for day 2020-05-19
Fetching keyword iPad for day 2020-05-19
Fetching keyword Macbook for day 2020-05-19
Fetching keyword iOS for day 2020-05-19
Fetching keyword macOS for day 2020-05-19
Fetching keyword iPhone for day 2020-05-18
Fetching keyword iPad for day 2020-

Fetching keyword macOS for day 2020-03-30
Fetching keyword iPhone for day 2020-03-27
Fetching keyword iPad for day 2020-03-27
Fetching keyword Macbook for day 2020-03-27
Fetching keyword iOS for day 2020-03-27
Fetching keyword macOS for day 2020-03-27
Fetching keyword iPhone for day 2020-03-26
Fetching keyword iPad for day 2020-03-26
Fetching keyword Macbook for day 2020-03-26
Fetching keyword iOS for day 2020-03-26
Fetching keyword macOS for day 2020-03-26
Fetching keyword iPhone for day 2020-03-25
Fetching keyword iPad for day 2020-03-25
Fetching keyword Macbook for day 2020-03-25
Fetching keyword iOS for day 2020-03-25
Fetching keyword macOS for day 2020-03-25
Fetching keyword iPhone for day 2020-03-24
Fetching keyword iPad for day 2020-03-24
Fetching keyword Macbook for day 2020-03-24
Fetching keyword iOS for day 2020-03-24
Fetching keyword macOS for day 2020-03-24
Fetching keyword iPhone for day 2020-03-23
Fetching keyword iPad for day 2020-03-23
Fetching keyword Macbook for day 2

Fetching keyword iPhone for day 2020-01-31
Fetching keyword iPad for day 2020-01-31
Fetching keyword Macbook for day 2020-01-31
Fetching keyword iOS for day 2020-01-31
Fetching keyword macOS for day 2020-01-31
Fetching keyword iPhone for day 2020-01-30
Fetching keyword iPad for day 2020-01-30
Fetching keyword Macbook for day 2020-01-30
Fetching keyword iOS for day 2020-01-30
Fetching keyword macOS for day 2020-01-30
Fetching keyword iPhone for day 2020-01-29
Fetching keyword iPad for day 2020-01-29
Fetching keyword Macbook for day 2020-01-29
Fetching keyword iOS for day 2020-01-29
Fetching keyword macOS for day 2020-01-29
Fetching keyword iPhone for day 2020-01-28
Fetching keyword iPad for day 2020-01-28
Fetching keyword Macbook for day 2020-01-28
Fetching keyword iOS for day 2020-01-28
Fetching keyword macOS for day 2020-01-28
Fetching keyword iPhone for day 2020-01-27
Fetching keyword iPad for day 2020-01-27
Fetching keyword Macbook for day 2020-01-27
Fetching keyword iOS for day 202

Fetching keyword iPad for day 2019-12-04
Fetching keyword Macbook for day 2019-12-04
Fetching keyword iOS for day 2019-12-04
Fetching keyword macOS for day 2019-12-04
Fetching keyword iPhone for day 2019-12-03
Fetching keyword iPad for day 2019-12-03
Fetching keyword Macbook for day 2019-12-03
Fetching keyword iOS for day 2019-12-03
Fetching keyword macOS for day 2019-12-03
Fetching keyword iPhone for day 2019-12-02
Fetching keyword iPad for day 2019-12-02
Fetching keyword Macbook for day 2019-12-02
Fetching keyword iOS for day 2019-12-02
Fetching keyword macOS for day 2019-12-02
Fetching keyword iPhone for day 2019-11-29
Fetching keyword iPad for day 2019-11-29
Fetching keyword Macbook for day 2019-11-29
Fetching keyword iOS for day 2019-11-29
Fetching keyword macOS for day 2019-11-29
Fetching keyword iPhone for day 2019-11-27
Fetching keyword iPad for day 2019-11-27
Fetching keyword Macbook for day 2019-11-27
Fetching keyword iOS for day 2019-11-27
Fetching keyword macOS for day 2019

Fetching keyword Macbook for day 2019-10-09
Fetching keyword iOS for day 2019-10-09
Fetching keyword macOS for day 2019-10-09
Fetching keyword iPhone for day 2019-10-08
Fetching keyword iPad for day 2019-10-08
Fetching keyword Macbook for day 2019-10-08
Fetching keyword iOS for day 2019-10-08
Fetching keyword macOS for day 2019-10-08
Fetching keyword iPhone for day 2019-10-07
Fetching keyword iPad for day 2019-10-07
Fetching keyword Macbook for day 2019-10-07
Fetching keyword iOS for day 2019-10-07
Fetching keyword macOS for day 2019-10-07
Fetching keyword iPhone for day 2019-10-04
Fetching keyword iPad for day 2019-10-04
Fetching keyword Macbook for day 2019-10-04
Fetching keyword iOS for day 2019-10-04
Fetching keyword macOS for day 2019-10-04
Fetching keyword iPhone for day 2019-10-03
Fetching keyword iPad for day 2019-10-03
Fetching keyword Macbook for day 2019-10-03
Fetching keyword iOS for day 2019-10-03
Fetching keyword macOS for day 2019-10-03
Fetching keyword iPhone for day 20